In [1]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.exceptions import ConvergenceWarning
from functions.formatting import get_subgroup_str
from itertools import product, combinations

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
omit_demographics = True 
use_gerryfair = False 

In [3]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [5]:
X = df.drop('completed', axis=1)
y = df['completed']

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [6]:
demo_groups = [['US', 'intl'], ['bachelor_obtained', 'no_bachelor_obtained', 'education_na'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']]

In [7]:
def compute_combos():
    ret = []
    for i in range(1,len(demo_groups)+1):
        g = list(combinations(demo_groups, i))
        for demos in g:
            ret += list(product(*demos))

    return ret

In [8]:
combos = compute_combos()

In [9]:
combos.sort(key=len)
combos

[('US',),
 ('intl',),
 ('bachelor_obtained',),
 ('no_bachelor_obtained',),
 ('education_na',),
 ('white',),
 ('black',),
 ('asian',),
 ('latinx',),
 ('race_others',),
 ('race_na',),
 ('male',),
 ('female',),
 ('gender_na',),
 ('gender_other',),
 ('US', 'bachelor_obtained'),
 ('US', 'no_bachelor_obtained'),
 ('US', 'education_na'),
 ('intl', 'bachelor_obtained'),
 ('intl', 'no_bachelor_obtained'),
 ('intl', 'education_na'),
 ('US', 'white'),
 ('US', 'black'),
 ('US', 'asian'),
 ('US', 'latinx'),
 ('US', 'race_others'),
 ('US', 'race_na'),
 ('intl', 'white'),
 ('intl', 'black'),
 ('intl', 'asian'),
 ('intl', 'latinx'),
 ('intl', 'race_others'),
 ('intl', 'race_na'),
 ('US', 'male'),
 ('US', 'female'),
 ('US', 'gender_na'),
 ('US', 'gender_other'),
 ('intl', 'male'),
 ('intl', 'female'),
 ('intl', 'gender_na'),
 ('intl', 'gender_other'),
 ('bachelor_obtained', 'white'),
 ('bachelor_obtained', 'black'),
 ('bachelor_obtained', 'asian'),
 ('bachelor_obtained', 'latinx'),
 ('bachelor_obtained

In [10]:
demographics = ['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
protected=['US', 'intl', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [11]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=omit_demographics, demographics=demographics, protected=protected, is_gerryfair=True, iters=3)

/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


iteration: 1
most accurate classifier accuracy: 0.21988950276243094, most acc-class unfairness: 0.027888541916886864, most acc-class size 0.3425414364640884
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.20331491712707184, most acc-class unfairness: 0.02558436039099023, most acc-class size 0.36022099447513817
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.2229580573951435, most acc-class unfairness: 0.029284976645918014, most acc-class size 0.17660044150110377
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.20750551876379691, most acc-class unfairness: 0.022822210901681107, most acc-class size 0.33995584988962474
iteration: 2
iteration: 1
most accurate classifier accuracy: 0.22185430463576158, most acc-class unfairness: 0.03213227605437435, most acc-class size 0.33002207505518766
iteration: 2


/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/sarthakagrawal/Documents/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rco

In [24]:
res

[{'subgroup': 'US',
  'n': '201.4',
  'auc_avg': '0.773',
  'auc_std': '0.024',
  'fpr_avg': '0.185',
  'fpr_std': '0.027',
  'rmse_avg': '0.478',
  'rmse_std': '0.023'},
 {'subgroup': 'intl',
  'n': '25.0',
  'auc_avg': '0.735',
  'auc_std': '0.076',
  'fpr_avg': '0.074',
  'fpr_std': '0.065',
  'rmse_avg': '0.440',
  'rmse_std': '0.136'},
 {'subgroup': 'bachelor_obtained',
  'n': '103.8',
  'auc_avg': '0.797',
  'auc_std': '0.036',
  'fpr_avg': '0.180',
  'fpr_std': '0.058',
  'rmse_avg': '0.452',
  'rmse_std': '0.043'},
 {'subgroup': 'no_bachelor_obtained',
  'n': '12.2',
  'auc_avg': '0.656',
  'auc_std': '0.088',
  'fpr_avg': '0.018',
  'fpr_std': '0.036',
  'rmse_avg': '0.416',
  'rmse_std': '0.063'},
 {'subgroup': 'education_na',
  'n': '110.4',
  'auc_avg': '0.748',
  'auc_std': '0.035',
  'fpr_avg': '0.181',
  'fpr_std': '0.026',
  'rmse_avg': '0.501',
  'rmse_std': '0.032'},
 {'subgroup': 'white',
  'n': '76.4',
  'auc_avg': '0.750',
  'auc_std': '0.052',
  'fpr_avg': '0.324'

Generate a list of all subgroup data and write it to a csv

In [ ]:
csv_file = f"./data/MTC508_subgroup_data_gerryfair_with_all.csv"
# csv_file = "test.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [ ]:
demo_groups = [['country_cd_US'], ['bachelor_obtained'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_other', 'gender_na']]

In [ ]:
data = {}

data['overall'] = len(X)

for c in combos[1:]:
    masks = [X[name] == value for name, value in c]
    final_mask = pd.concat(masks, axis=1).all(axis=1)
    filter_X = X[final_mask]
    data[get_subgroup_str(c)] = len(filter_X)




In [ ]:
data

In [ ]:
df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# Specify the CSV file path
csv_file_path = 'data/MTC508_data_counts.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

In [ ]:
data = []
for i in range(2,11):
    data.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=i))

In [ ]:
data

In [ ]:
fprs = []
for res in data:
    for subgroup in res:
        if subgroup['subgroup'] == 'Overall':
            fprs.append(float(subgroup['fpr_avg']))
            break

In [ ]:
fprs = np.array(fprs)
fprs.size

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

iterations = np.arange(1, 10)

other_algorithm_fpr = np.linspace(0.5, 0.5, 9) 

plt.plot(iterations, fprs, marker='o', linestyle='-', label='GerryFair')

plt.plot(iterations, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

plt.xlabel('Iterations')
plt.ylabel('False Positive Rate (FPR)')
plt.title('Comparison of Algorithms - FPR vs Iterations')

plt.legend()

plt.show()

In [ ]:
fprs

In [ ]:
demographics = ['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [ ]:
default_fprs = {"Overall": 0.157, "black": 0.360, "female": 0.177, "black, female": 0.537}


In [ ]:
def build_graph_iters(start, stop, step, attrs):
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=i))
    

    for a in attrs:
        fprs = []
        for res in data:
            
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], iterations.size) 

        plt.plot(iterations, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(iterations, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Iterations')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Iterations For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_iters(20,500,20,['Overall', 'black', 'female', 'black, female'])

In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [ ]:
len(data)

In [ ]:
def build_graph_attrs(iters, attrs):
    results = []
    for i in range(len(demographics)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_attrs(5, ['Overall', 'black', 'female', 'black, female'])